# LOAD DATA

In [ ]:
!pip install tensorflow
!pip install pandas

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

In [4]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [5]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")


2194/2194 ━━━━━━━━━━━━━━━━━━━━ 0s 3us/step
573/573 ━━━━━━━━━━━━━━━━━━━━ 0s 10us/step


In [6]:
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)


In [7]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [8]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3



# INPUT FUNCTION


In [9]:
def input_fn(features, labels, training=True, batch_size=256):
  # convert the inputs to a Dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  # shuffle and repeat if you are in training mode
  if training:
    dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

# FEATURE COLUMNS


In [ ]:
# feature columns describe how to use the input
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

# BULDING THE MODEL

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

# One-hot encode the labels
train_y = to_categorical(train_y, num_classes=3)
test_y = to_categorical(test_y, num_classes=3)

# Build the DNN model
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(len(train.keys()),)),  # Input layer
    layers.Dense(64, activation='relu'),  # Hidden layer 1
    layers.Dense(32, activation='relu'),  # Hidden layer 2
    layers.Dense(3, activation='softmax')  # Output layer with 3 units for 3 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train, train_y, epochs=50, batch_size=32)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test, test_y)

In [12]:
print(f"Test accuracy: {test_accuracy}")

Test accuracy: 0.9666666388511658


# PREDICTIONS

In [13]:
import numpy as np

new_flower = pd.DataFrame({
    'SepalLength': [5.9],
    'SepalWidth': [3.0],
    'PetalLength': [4.2],
    'PetalWidth': [1.5]
})

# Predict the species
predictions = model.predict(new_flower)

class_id = np.argmax(predictions, axis=1)[0]  # Get the index of the highest probability class
probability = predictions[0][class_id]  # Get the highest probability

# Map the predicted class index to the species name
predicted_species = SPECIES[class_id]

# Print the result with the confidence percentage
print('Prediction is "{}" ({:.1f}%)'.format(predicted_species, 100 * probability))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Prediction is "Versicolor" (93.6%)
